In [68]:
import numpy as np
import pandas

def d20():
    return np.random.randint(1,21)

def do_rounds_and_update(a,b,rounds,df):
    wins = do_match(a,b,rounds)
    a_wins = wins[0]
    b_wins = wins[1]
    a_id = a[0]
    b_id = b[0]
    df.at[a_id,'total_wins'] = df.at[a_id,'total_wins'] + a_wins
    df.at[b_id,'total_wins'] = df.at[b_id,'total_wins'] + b_wins
    if a_wins > b_wins:
        df.at[a_id,'pair_wins'] = df.at[a_id,'pair_wins'] + 1
    else:
        df.at[b_id,'pair_wins'] = df.at[b_id,'pair_wins'] + 1
    return creatures

def do_match(a,b,rounds):
    a_wins = 0
    b_wins = 0
    for _ in np.arange(rounds):
        wins = do_round(a,b)
        a_wins = a_wins + wins[0]
        b_wins = b_wins + wins[1]
    print ("After %d rounds %s won %d times and %s won %d times" % (rounds,a.name,a_wins,b.name,b_wins))
    return [a_wins,b_wins]

def do_round( a, b ):
    a_init = d20() + a.init
    if debug:
        print("%s rolled %d init" % (a.name,a_init))
    b_init = d20() + b.init
    if debug:
        print("%s rolled %d init" % (b.name,b_init))
    if (a_init >= b_init):
        first = a.copy()
        second = b.copy()
        in_order = True
    else:
        first = b.copy()
        second = a.copy()
        in_order = False
    round = 0
    while (first.hp > 0 and second.hp > 0):
        if debug:
            print("    Round %d: %s hp is %d and %s hp is %d" % (round+1, first.name,first.hp,second.name,second.hp))
        round = round + 1
        first,second = do_attack(first,second)
        if (second.hp <= 0):
            if debug:
                print("%s is dead" % second.name)
            if in_order:
                return [1,0] #a went first and won
            else:
                return [0,1] #b went first and won
        else:
            second,first = do_attack(second,first)
            if (first.hp <= 0):
                if debug:
                    print("%s is dead" % first.name)
                if in_order:
                    return [0,1] #b went second and won
                else:
                    return [1,0] #a went second and won
    """Something went wrong here, nobody wins"""
    return [0,0]
    
def do_attack( attacker, target ):
    if debug:
        print ("        %s makes %d attacks against %s" % (attacker.name, attacker.att_num, target.name))
    for _ in np.arange(attacker.att_num):
        attack = d20() + attacker.hit_mod
        if (attack >= target.ac):
            target.hp = target.hp - attacker.dam_avg
            if debug:
                print("            %s hits %s for %d damage" % (attacker.name, target.name, attacker.dam_avg ))
        else:
            if debug:
                print("            %s misses %s" % (attacker.name, target.name))
    return [attacker, target]

In [82]:
debug = False
rounds = 300
creatures = pandas.read_csv('creatures.csv')
rows = creatures.shape[0]
creatures['pair_wins'] = np.zeros(rows)
creatures['total_wins'] = np.zeros(rows)
a = creatures.sample(1).to_records()[0]
b = creatures.sample(1).to_records()[0]
#a = creatures.take([0]).to_records()[0]
#b = creatures.take([1]).to_records()[0]
creatures = do_rounds_and_update(a,b,rounds,creatures)
creatures.loc[creatures['total_wins'] > 0]

After 300 rounds Giant Ape won 64 times and Frost Giant won 236 times


,name,cr,ac,hp,init,att_num,hit_mod,dam_avg,pair_wins,total_wins
109,Frost Giant,8.0,15,138,-1.90,2,3,5.0,1.0,236.0
115,Giant Ape,7.0,12,157,2.14,2,0,7.0,0.0,64.0
